In [13]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import os


In [14]:
DB_HOST = "localhost"  # Local machine since Jupyter runs locally
DB_NAME = "salesdb"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_PORT = "5432"

def get_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT
    )


In [15]:
csv_path = os.path.join("..", "data", "sales_raw.csv")
df = pd.read_csv(csv_path)
df.head()


,ReportID,SalesPersonID,ProductID,Quantity,TotalSalesValue,Suspicious
0,Rep10101,C21116,PR6112,182,1665,indeterminate
1,Rep10102,C21116,PR6112,182,1740,indeterminate
2,Rep10103,C21116,PR6253,101,1040,indeterminate
3,Rep10104,C21116,PR6253,283,4495,No
4,Rep10105,C21116,PR6294,108,1465,indeterminate


In [16]:
df.columns


Index(['ReportID', 'SalesPersonID', 'ProductID', 'Quantity', 'TotalSalesValue',
       'Suspicious'],
      dtype='object')

In [17]:
df = df.rename(columns={
    "ReportID": "report_id",
    "SalesPersonID": "salesperson_id",
    "ProductID": "product_id",
    "Quantity": "quantity",
    "TotalSalesValue": "total_sales",
    "Suspicious": "suspicious",
})

df.head(), df.columns


(  report_id salesperson_id product_id  quantity  total_sales     suspicious
 0  Rep10101         C21116     PR6112       182         1665  indeterminate
 1  Rep10102         C21116     PR6112       182         1740  indeterminate
 2  Rep10103         C21116     PR6253       101         1040  indeterminate
 3  Rep10104         C21116     PR6253       283         4495             No
 4  Rep10105         C21116     PR6294       108         1465  indeterminate,
 Index(['report_id', 'salesperson_id', 'product_id', 'quantity', 'total_sales',
        'suspicious'],
       dtype='object'))

In [18]:
create_tables_sql = """
CREATE TABLE IF NOT EXISTS sales_raw_kaggle (
    report_id       VARCHAR(50),
    salesperson_id  VARCHAR(50),
    product_id      VARCHAR(50),
    quantity        INTEGER,
    total_sales     NUMERIC(14,2),
    suspicious      VARCHAR(20)
);

CREATE TABLE IF NOT EXISTS sales_by_salesperson (
    salesperson_id   VARCHAR(50) PRIMARY KEY,
    total_quantity   INTEGER,
    total_sales      NUMERIC(16,2),
    num_transactions INTEGER,
    num_suspicious   INTEGER
);
"""

with get_connection() as conn:
    with conn.cursor() as cur:
        cur.execute(create_tables_sql)
    conn.commit()

print("Tables created (if they didn't already exist).")


Tables created (if they didn't already exist).


In [19]:
# Make sure numeric columns have the right type
df["quantity"] = df["quantity"].astype(int)
df["total_sales"] = df["total_sales"].astype(float)

cols = ["report_id", "salesperson_id", "product_id", "quantity", "total_sales", "suspicious"]
records = df[cols].to_records(index=False)
rows = list(records)
len(rows)


42582

In [20]:
# Ensure correct dtypes in pandas
df["quantity"] = df["quantity"].astype(int)
df["total_sales"] = df["total_sales"].astype(float)

# Build rows as pure Python types (no numpy types)
rows = [
    (
        str(row.report_id),
        str(row.salesperson_id),
        str(row.product_id),
        int(row.quantity),
        float(row.total_sales),
        str(row.suspicious),
    )
    for row in df.itertuples(index=False)
]

len(rows)



42582

In [21]:
from psycopg2.extras import execute_values

insert_query = """
    INSERT INTO sales_raw_kaggle (
        report_id,
        salesperson_id,
        product_id,
        quantity,
        total_sales,
        suspicious
    ) VALUES %s
"""

with get_connection() as conn:
    with conn.cursor() as cur:
        execute_values(cur, insert_query, rows)
    conn.commit()

print(f"Inserted {len(rows)} rows into sales_raw_kaggle.")


Inserted 42582 rows into sales_raw_kaggle.


In [22]:
with get_connection() as conn:
    preview_df = pd.read_sql("SELECT * FROM sales_raw_kaggle LIMIT 5;", conn)

preview_df


C:\Users\cheth\AppData\Local\Temp\ipykernel_24644\351140156.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preview_df = pd.read_sql("SELECT * FROM sales_raw_kaggle LIMIT 5;", conn)


,report_id,salesperson_id,product_id,quantity,total_sales,suspicious
0,Rep10101,C21116,PR6112,182,1665.0,indeterminate
1,Rep10102,C21116,PR6112,182,1740.0,indeterminate
2,Rep10103,C21116,PR6253,101,1040.0,indeterminate
3,Rep10104,C21116,PR6253,283,4495.0,No
4,Rep10105,C21116,PR6294,108,1465.0,indeterminate


In [23]:
aggregation_sql = """
INSERT INTO sales_by_salesperson (
    salesperson_id,
    total_quantity,
    total_sales,
    num_transactions,
    num_suspicious
)
SELECT
    salesperson_id,
    SUM(quantity) AS total_quantity,
    SUM(total_sales) AS total_sales,
    COUNT(*)      AS num_transactions,
    SUM(
        CASE
            WHEN suspicious <> 'No' THEN 1
            ELSE 0
        END
    ) AS num_suspicious
FROM sales_raw_kaggle
GROUP BY salesperson_id
ON CONFLICT (salesperson_id) DO UPDATE
SET
    total_quantity   = EXCLUDED.total_quantity,
    total_sales      = EXCLUDED.total_sales,
    num_transactions = EXCLUDED.num_transactions,
    num_suspicious   = EXCLUDED.num_suspicious;
"""

with get_connection() as conn:
    with conn.cursor() as cur:
        cur.execute(aggregation_sql)
    conn.commit()


In [24]:
with get_connection() as conn:
    summary_df = pd.read_sql(
        "SELECT * FROM sales_by_salesperson ORDER BY total_sales DESC LIMIT 10;",
        conn
    )
summary_df


C:\Users\cheth\AppData\Local\Temp\ipykernel_24644\2377699905.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  summary_df = pd.read_sql(


,salesperson_id,total_quantity,total_sales,num_transactions,num_suspicious
0,C21178,14714598,53045800.0,706,706
1,C22185,2347174,51598220.0,794,722
2,C22030,11050508,41411720.0,1050,1012
3,C21954,3339894,40043830.0,144,138
4,C21453,5081726,34385240.0,766,718
5,C22214,1514006,34270260.0,486,424
6,C21872,15463732,32876330.0,586,530
7,C21636,48121902,31711640.0,976,956
8,C21829,1064918,28721272.0,562,492
9,C21864,855554,24528900.0,870,832
